In [ ]:
!pip install wikipedia

In [76]:
import numpy as np
import pandas as pd
from operator import eq
import matplotlib.pyplot as plt
plt.style.use(style='seaborn')
%matplotlib inline

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

In [78]:
#removing punctuations
#library that contains punctuation
import string
string.punctuation
!pip install stemming
from stemming.porter2 import stem

In [79]:
#defining the function to remove punctuation
def remove_punctuation(text):
  if(type(text)==float):
    return text
  ans=""  
  for i in text:     
    if i not in string.punctuation:
      ans+=i    
  return ans

In [80]:
def generate_Ngrams(text,ngram=1):
  word1=[word for word in text.split(" ") if word not in set(stopwords.words('english'))] 
  words=[] 
  for i in word1:
    words.append(lemmatizer.lemmatize(stem(i)))

  temp=zip(*[words[i:] for i in range(0,ngram)])
  ans=[' '.join(ngram) for ngram in temp]
  return ans

In [81]:
import wikipedia
result1 = wikipedia.search("Neural Networks")
result2 = wikipedia.search("guitar")
result3 = wikipedia.search("language")
results = result1+result2+result3
results.remove('Guitar Hero')
results.remove("Warr Guitar")
results.remove('Language (magazine)')

In [82]:
def remove_punctuation(text):
  if(type(text)==float):
    return text
  ans=""  
  for i in text:     
    if i not in string.punctuation:
      ans+=i    
  return ans

In [83]:
def calculator(l1):
  n=len(l1)
  m = np.zeros((n+1,n+1)) # numpy.zeros(shape, dtype = None, order = 'C')
  n=len(l1)
  for i in range(n):
    for j in range(i+1,n):
      list1=l1[i]
      list2=l1[j]
      # print(list1,list2)
      c=list(set(list1) & set(list2))
      res=len(c)
      num=max(len(list1),len(list2))
      res = res/num
      m[i][j]=res
      m[j][i]=res
  return m

def CalMatrix(final_list):
  n=len(final_list)
  # print(n)
  Ngram_list=[]

  for i in final_list:
    Ngram_list.append(generate_Ngrams(i,2))
    # print(Ngram_list)
   
  return calculator(Ngram_list)


In [84]:
def pred_order_normal(Matrix):
  final_order_lst = [0]
  for i in range(Matrix.shape[0]):
    sorted_tuples = []
    sorted_tuples = sorted(enumerate(Matrix[final_order_lst[-1]]), key = lambda i:i[1])
    # print(sorted_tuples)
    for elem in sorted_tuples[::-1]:
      if elem[0] not in final_order_lst:
        final_order_lst.append(elem[0])
  return final_order_lst

In [85]:
def get_pred_order_lookahead(Matrix):
  final_order_lst = [0]

  for i in range(Matrix.shape[0]):

    sorted_tuples = []

    prob_1 = 0
    prob_2 = 0
   
    sorted_tuples = sorted(enumerate(Matrix[final_order_lst[-1]]), key=lambda i: i[1])
   
    order_lst1 = final_order_lst.copy()
    order_lst2 = final_order_lst.copy()
   
    for elem in sorted_tuples[::-1]:
      if elem[0] not in order_lst1:
        order_lst1.append(elem[0])
        prob1 = elem[1]
        prob_1 = prob1
        break

    for elem in sorted_tuples[::-1]:
      if elem[0] not in order_lst2 and elem[0]!=order_lst1[-1]:
        order_lst2.append(elem[0])
        prob2 = elem[1]
        prob_2 = prob2
        break
    
    sorted_tuples1 = sorted(enumerate(Matrix[order_lst1[-1]]), key = lambda i: i[1])
    sorted_tuples2 = sorted(enumerate(Matrix[order_lst2[-1]]), key = lambda i: i[1])

    for elem in sorted_tuples1[::-1]:
      if elem[0] not in order_lst1:
        order_lst1.append(elem[0])
        prob11 = elem[1]
        prob_1 = prob1*prob11
        break

    for elem in sorted_tuples2[::-1]:
      # if elem[0] not in order_lst2 and elem[0]!=order_lst1[-1]:
      if elem[0] not in order_lst2:
        order_lst2.append(elem[0])
        prob22 = elem[1]
        prob_2 = prob2*prob22
        break

    if prob_1 > prob_2:
      final_order_lst = order_lst1[:-1:]

    else:
      final_order_lst = order_lst2[:-1:]

  for i in range(Matrix.shape[0]):
    if i not in final_order_lst:
      final_order_lst.append(i)
      break  
  
  return final_order_lst

In [86]:
def Ordered_output(final_order_lst,final_content_list):
  output_list=[]
  for i in final_order_lst:
    output_list.append(final_content_list[i])
  return output_list

In [87]:
def accuracy(lst):
  total = 0
  correct = 0
  for i in range(len(lst)-1):
    if (lst[i+1] - lst[i] == 1): 
      correct+=1
    # elif (lst[i] - lst[i+1] == 1):
    #   correct+=1
    total+=1
  return correct/total

In [91]:
def PageOrder(result):
  sze=len(result)
  AccuracyLst=[]

  for i in range(sze):
    page = wikipedia.page(result[i])
    content = page.content
    content_list = content.split("\n")
    
    final_content_list = []
    for j in range (len(content_list)):
      if content_list[j] and len(content_list[j].split(" ")) > 30:
        final_content_list.append(content_list[j])

    final_list=[]
    for j in final_content_list: 
      final_list.append(remove_punctuation(j))

    Matrix=CalMatrix(final_list)

    pred_order = get_pred_order_lookahead(Matrix)

    Acc_score=accuracy(pred_order)

    AccuracyLst.append(Acc_score)
    
  return AccuracyLst


In [92]:
def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
Alist=PageOrder(results)
Average(Alist)